# 14 - Prueba de Concepto: Tendencias Políticas en Twitter - Explotación -


* Este notebook es la continuación del notebook: *13_PoC_Tendencias_Politicas_Twitter_Generacion_Exportacion_Modelos.ipynb*


* En este notebook vamos a ***explotar*** un modelo ya creado anteriormente y va a tener como objetivo ***clasificar los tweets de una determinada cuenta de twitter*** en función de su tendencia política.


* Para realizar todo esto, lo haremos de la siguiente manera:

    1. Lectura (via API) de los tweets de una determinada cuenta de twitter
    2. Normalización de los tweets
    3. Importación de los modelos (Clasificación y BoW)
    3. Creacción de la Bolsa de Palabras (BoW) de los nuevos tweets
    4. Predicción
    
    
<hr>


## Lectura (via API) de tweets

* Para leer los tweets de una cuenta de twitter podemos usar el API de Twitter directamente o utilizar la librería ***tweepy*** que nos facilitamo mucho la labor a la hora de obtener datos de Twitter.


* No tenemos como objetivo en este notebook explicar como funciona esta librería. Para saber de su funcionamiento podeis ver su página web: https://www.tweepy.org/


* En esta punto vamos a leer 'N' tweets (si Twitter nos los facilita) de una determinada cuenta de Twitter.


* En primer lugar tenemos que autenticarnos en Twitter con el protocolo OAuth (https://es.wikipedia.org/wiki/OAuth) y para ello necesitamos unos keys y unos tokens que nos proporcionará Twitter al registrar una APP. Este proceso de registro de una App es un poco tediodo y tampoco es el objetivo en esta PoC el explicar ese proceso. Para más información visitar la web de desarrolladores de Twitter (https://developer.twitter.com/).


* Nos autenticamos con Twitter:


In [1]:
import tweepy
# Claves de cliente y tokens de acceso
CONSUMER_KEY = 'fcCFQpt3lhMzeCgGhznWsb8C5'
CONSUMER_SECRET = 'lljbTkudnEvn0SWn6ZPw5Svam6TzD9q58AhBsgrsqGN9AdqrvF'
ACCESS_TOKEN = '1021745678974902279-7VhCtZkPsqhcaHPq4NFalaShazGbTV'
ACCESS_TOKEN_SECRET = '0xDVjwimkp1WKjjZpMpLuj0V3aRkAeKuvc4DiHkyfvYkZ'

# Proceso de autenticación OAuth
auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET)
auth.set_access_token(ACCESS_TOKEN, ACCESS_TOKEN_SECRET)
api = tweepy.API(auth, wait_on_rate_limit=False)

* Vamos a petir a Twitter que nos devuelva de una determinada cuenta (*account*) un determinado número de tweets (*num_tweets*)


* Dado que se ha registrado una aplicación de Twitter gratuita, es posible que Twitter no nos devuelva de una determinada cuenta todos los tweets que le pedimos.


* A modo de poder probar esta prueba de concepto, dejo algunas cuentas de Twitter (de marcado caracter político) para predecir cual es su tendencia política. Esta cuentas no han sido utilizadas para generar el dataset con el que hemos generado el modelo.

    - Iñigo Errejon: @ierrejon
    - Manuela Carmena: @ManuelaCarmena
    - Susana Díaz: @susanadiaz
    - Josep Borrell: @JosepBorrellF
    - Mariano Rajoy: @marianorajoy
    - Jordi Evole: @jordievole
    - Eduardo Inda: @eduardoinda
    - Jorge Verstrynge: @VerstryngeJorge

In [2]:
account = '@ierrejon'
num_tweets = 50
tweets = list()
for user_status in api.user_timeline(screen_name=account, count = num_tweets, include_rts=False, tweet_mode="extended"):
        tweets.append(user_status.full_text)

for index, tweet in enumerate(tweets):
    print('\n{} - {}'.format(index, tweet))


0 - @DavidJarR Tal cual

1 - @carla_varona @Panchovarona @ObjetivoLaSexta @_anapastor_ ☺️

2 - @Panchovarona @ObjetivoLaSexta @_anapastor_ Gracias Pancho!!!

3 - @YaizaYgarcia Gracias a ti por decirlo!

4 - @MiguelAML2 Muchas gracias , Miguel !

5 - @NisaAlberto Gracias!!

6 - Apostar por energías verdes es reducir la factura de la luz. https://t.co/4MvnxEWX8A

7 - @tardedetortasf2 A ti! 😊

8 - Para que la crisis no la paguen los de siempre: https://t.co/bEs05d5EXX

9 - @tiguel_mena @Caquel__ @gabrielrufian ¿En esta, dices? https://t.co/vE4EAP5jad

10 - El elefante en el hemiciclo. https://t.co/IzZwMdOhHV

11 - @Alexa4u2 😊✌🏼

12 - @CuetoMartos Las comas, por favor.

13 - @RTalaveraM2 Gracias!!

14 - Todo el día con “España” en la boca y a la hora de la verdad van con las eléctricas y las petroleras antes que con su país. https://t.co/kN5Uql25bp

15 - @Juanmahuerta Pues tienes toda la razón . Gracias por decirlo .

16 - El mercado: hecha la ley, hecha la trampa https://t.co/qygQEVNu15


<hr>


## Normalización


* Tenemos que normalizar de la misma manera que se han normlizado los tweets con los que se generaron el modelo.


* Realizamos las misma acciones para ***normalizar*** los tweets:
    1. Pasamos las frases a minúsculas.
    2. Sustituimos los puntos por espacios ya que hay muchas palabras unidas por un punto
    3. Quitamos la almuhadilla de los hashtags para considerarlos como palabras.
    4. Eliminamos los signos de puntuación.
    5. Eliminamos las palabras con menos de 3 caracteres.
    6. Eliminamos las Stop-Words.
    7. Eliminamos los enlaces(http) y las menciones (@)
    8. Pasamos la palabra a su lema
    

* Por último vamos a eliminar los tweets que tras la normalización no contengan ninguna palabra.

In [3]:
import spacy

from tqdm import tqdm

nlp = spacy.load('es_core_news_sm')

def normalize(tweets, min_words=3):
    """
    Función que dada una lista de tweets ([tweet, clase]), normaliza los tweets y devuelve una lista
    con los tweets normalizados, descartando aquellos tweets que tras la normalización tengan menos de
    "min_words" palabras en el tweet.
    
    :param tweets:       Lista de Tweets ([tweet, clase]) con el tweet y la clase a la que pertenece
    :param min_words:    Número minimo de palabras que tiene que tener un tweet tras la normalización
    :return:             Lista de Tweets ([tweet, clase]) normalizados
    """
    tweets_list = []
    for tweet in tqdm(tweets):
        # Tokenizamos el tweets realizando los puntos 1,2 y 3.
        tw = nlp(tweet.lower().replace('.', ' ').replace('#', ' ').strip())
        
        # Normalizamos Puntos 4,5,6,7 y 8
        tw = ([word.lemma_ for word in tw if (not word.is_punct)
               and (len(word.text) > 2) and (not word.is_stop)
               and (not word.text.startswith('@'))
               and (not word.text.startswith('http'))
               and (not ':' in word.text)])
        
        # Eliminamos los tweets que tras la normalización tengan menos de "min_words" palabras
        if len(tw) >= min_words:
            tweets_list.append(" ".join(tw))
    return tweets_list

# Normalizamos las frases
X_norm = normalize(tweets, min_words=5)

print('Nº de Tweets validos de {}: {}'.format(account, len(X_norm)))

100%|█████████████████████████████████████████████████████████████████████████████████| 37/37 [00:00<00:00, 179.41it/s]

Nº de Tweets validos de @ierrejon: 14


<hr>


## Importamos los modelos

* Vamos a importar los modelos creados en el notebook *13_PoC_Tendencias_Politicas_Twitter_Generacion_Exportacion_Modelos.ipynb*


* Temos que importar:
    1. El modelo creado con el Algoritmo de Aprendizaje ***Bernoulli Naive Bayes***
    2. El modelo para crear la Bolsa de Palabras
    
    
#### 1. Importamos el modelo para la clasificación

In [4]:
import pickle
filename = './models/13_best_model_tweets_politica.pickle'
classifier_model = pickle.load(open(filename, 'rb'))

#### 2. Importamos el modelo para crear la Bolsa de Palabras

In [5]:
filename = './models/13_vectorizer_bow_tweets_politica.pickle'
vectorizer = pickle.load(open(filename, 'rb'))

<hr>


## Creamos la Bolsa de Palabras


* Tenemos que usar el modelo de bolsa de palabras creado con los tweets de entrenamiento ya que contiene el diccionario (o vocabulario) con el que se ha entrenado el modelo para la clasificación.


* Este diccionario contiene 1000 palabras y los tweets que tenemos que predecir hay que transformalos a un vector de frecuencias donde nos diga cuantas veces aparecen las palabras del tweets dentro de esas 1000 palabras con las que hemos entrenado.


* En el caso de que un tweet contenga una palabra que no esté entre esas 1000 palabras, esta palabra no se tendrá en cuenta para predecir la tendencia política del tweet, ya que el modelo no ha sido entrenado con esa palabra.


* Vamos a pasar ***Bolsa de Palabras de frecuencias*** los tweets leidos

In [6]:
X_bow = vectorizer.transform(X_norm)

<hr>


## Predicción


* Con el modelo importado y la bolsa de palabras creada, vamos a clasificar cada unos de los tweets en su tendencia política.


* Para ello vamos a llamar al método "***predict()***" y le vamos a pasar la ***Bolsa de palabras de los Tweets*** para que nos clasifique ese tweet.


* Como lo que nos interesa es ***clasificar la cuenta de Twitter*** (o persona que esta detras de esa cuenta) en su ***tendencia política***, vamos a calcular los porcentajes de clasificación de los tweets en su tendencia política:

In [7]:
predictions = classifier_model.predict(X_bow)
prediccion = dict()
total = 0
for pred in predictions:
    if pred in prediccion:
        prediccion[pred] += 1
    else:
        prediccion[pred] = 1
    total += 1
        
for k,v in prediccion.items():
    print('{partido} = {porc:0.1f}%'.format(partido=k, porc=(v/float(total))*100))

podemos = 35.7%
vox = 28.6%
pp = 7.1%
psoe = 28.6%
